# 데이터 정제 및 준비

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 누락 데이터 처리하기

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

In [ ]:
string_data[0] = None #Numpy의 상수
string_data.isnull()

### 누락 데이터 골라내기 : dropna()

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()] #null이 아닌값만 True로 되어서 출력됨

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna() #nan을 뜻하는 NA값이 하나라도 있으면 다 nan으로 변경함

In [ ]:
data

In [ ]:
cleaned

In [ ]:
data.dropna(how='all') #전부 널인것만 지울때

In [ ]:
data[4] = NA
data

In [ ]:
data.dropna(axis=1, how='all') #열값이 Null 인것만 지우는 경우

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [ ]:
df.dropna()

In [ ]:
# 몇 개의 이상값이 들어있는 로우만 볼 경우
df.dropna(thresh=2)

### 결측치 채우기: fillna()

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
df.fillna(df.mean())

## 데이터 변형

### 중복 제거: drop_duplicates()

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
#k1 컬럼에 기반해서 중복 제거
data.drop_duplicates(['k1'])

In [ ]:
# 중복값 중, 마지막으로 발견된 값을 유지
data.drop_duplicates(['k1', 'k2'], keep='last')

### 함수나 매핑을 이용해서 데이터 변형하기 : map()

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
data['animal'] = data['food'].str.lower().map(meat_to_animal) 
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### 값 치환하기 : replace()

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### 컬럼/색인 이름 바꾸기: rename

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)
data

In [ ]:
data.index = data.index.map(transform)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### 개별화(Discretization) and 양자화(Binning)

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
bins = [18, 25, 35, 60, 100] #소괄호 : 초과,  대괄호 : 이상
cats = pd.cut(ages, bins)
cats

In [ ]:
cats.codes

In [ ]:
cats.categories

In [ ]:
pd.value_counts(cats)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) # right는 끝이 대괄호 이상,이하로 바뀜 . False = 처음과 끝을 포함

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

### 특잇값 찾고 제외하기

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

In [ ]:
data.describe()

In [ ]:
col = data[2]
col[np.abs(col) > 3]

In [ ]:
# 절대값이 3을 초과하는 값이 들어있는 모든 로우 선택
data[(np.abs(data) > 3).any(1)] #한개라도 3보다 큰지 확인

In [ ]:
# 절대값이 3을 초과하는 수는 3 또는 -3으로 설정
# sign(data) : 양수(1) or 음수(-1)인지 판단
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

## 연습문제

- 자유롭게 데이터전처리하기

    [예시]
    - 컬럼명 변경하기
    - 날짜 데이터 파싱하기
    - 이상치 처리하기
    - 계절, 날씨를 text로 변경하기(예: 봄, 여름, 가을, 겨울)
    - 자전거 대여횟수를 카테고리화 하기(예: 매우 적음, 적음, 보통, 많음, 매우 많음)
    - atemp 값을 소수점 2자리까지 보여지도록 바꾸기
    
    
- 데이터전처리 과정을 마친 데이터프레임을 csv 형태로 저장

- 코드제출

    https://drive.google.com/drive/folders/149Ryp8xzvMnklI4bY7kF074tSsr5-H0c?usp=sharing
    
    


In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# 읽어드릴 파일 경로 : /dataset/bike_sharing_demand_train.csv
filepath = "./datasets/exercise/bike_sharing_demand_train.csv"
data = pd.read_csv(filepath)
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


자유롭게 데이터전처리하기
## 컬럼명 변경하기

In [2]:
data.rename(columns={'datetime':'datatime(total)'})

,datatime(total),season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


## 날짜 데이터 파싱하기

In [3]:
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S', errors='raise')
data["year"] = data["datetime"].dt.year 
data["month"] = data["datetime"].dt.month 
data["day"] = data["datetime"].dt.day
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011,1,1
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011,1,1
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011,1,1
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012,12,19
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012,12,19
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012,12,19
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012,12,19


## 이상치 처리하기

In [4]:
data.isnull().sum() # 결측치 확인
data.describe() #
a = data["atemp"]>40   #극한의 날씨로 필터링
b = data["atemp"]<0
data[a|b]

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day
2245,2011-06-01 11:00:00,2,0,1,2,33.62,40.150,59,15.0013,26,86,112,2011,6,1
2246,2011-06-01 12:00:00,2,0,1,1,35.26,41.665,53,0.0000,32,137,169,2011,6,1
2247,2011-06-01 13:00:00,2,0,1,1,36.90,40.910,37,12.9980,29,125,154,2011,6,1
2248,2011-06-01 14:00:00,2,0,1,1,36.90,40.910,37,16.9979,40,105,145,2011,6,1
2249,2011-06-01 15:00:00,2,0,1,1,36.90,41.665,39,19.9995,25,127,152,2011,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,2012-08-04 16:00:00,3,0,0,1,36.90,40.910,39,19.9995,197,253,450,2012,8,4
8715,2012-08-05 12:00:00,3,0,0,1,35.26,40.150,47,35.0008,161,330,491,2012,8,5
8716,2012-08-05 13:00:00,3,0,0,1,36.08,40.910,44,31.0009,208,315,523,2012,8,5
8717,2012-08-05 14:00:00,3,0,0,1,36.90,42.425,42,31.0009,161,365,526,2012,8,5


## 계절, 날씨를 text로 변경하기(예: 봄, 여름, 가을, 겨울)

In [5]:
data["season"]=data["season"].replace([1,2,3,4],["봄","여름","가을","겨울"])
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day
0,2011-01-01 00:00:00,봄,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011,1,1
1,2011-01-01 01:00:00,봄,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011,1,1
2,2011-01-01 02:00:00,봄,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011,1,1
3,2011-01-01 03:00:00,봄,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011,1,1
4,2011-01-01 04:00:00,봄,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,겨울,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012,12,19
10882,2012-12-19 20:00:00,겨울,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012,12,19
10883,2012-12-19 21:00:00,겨울,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012,12,19
10884,2012-12-19 22:00:00,겨울,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012,12,19


## 자전거 대여횟수를 카테고리화 하기(예: 매우 적음, 적음, 보통, 많음, 매우 많음)

In [6]:
category_num = [0,50,200,400,600,1000]
category_text = ["매우 적음","적음","보통","많음","매우 많음"]
categroied_cut = pd.cut(data["count"],category_num,labels=category_text)
data["분석"]=categroied_cut
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,분석
0,2011-01-01 00:00:00,봄,0,0,1,9.84,14.395,81,0.0000,3,13,16,2011,1,1,매우 적음
1,2011-01-01 01:00:00,봄,0,0,1,9.02,13.635,80,0.0000,8,32,40,2011,1,1,매우 적음
2,2011-01-01 02:00:00,봄,0,0,1,9.02,13.635,80,0.0000,5,27,32,2011,1,1,매우 적음
3,2011-01-01 03:00:00,봄,0,0,1,9.84,14.395,75,0.0000,3,10,13,2011,1,1,매우 적음
4,2011-01-01 04:00:00,봄,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,1,1,매우 적음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,겨울,0,1,1,15.58,19.695,50,26.0027,7,329,336,2012,12,19,보통
10882,2012-12-19 20:00:00,겨울,0,1,1,14.76,17.425,57,15.0013,10,231,241,2012,12,19,보통
10883,2012-12-19 21:00:00,겨울,0,1,1,13.94,15.910,61,15.0013,4,164,168,2012,12,19,적음
10884,2012-12-19 22:00:00,겨울,0,1,1,13.94,17.425,61,6.0032,12,117,129,2012,12,19,적음


## atemp 값을 소수점 2자리까지 보여지도록 바꾸기

In [8]:
format = lambda x:'{:.2f}'.format(x) 
data['atemp']=data['atemp'].apply(format)
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,분석
0,2011-01-01 00:00:00,봄,0,0,1,9.84,14.39,81,0.0000,3,13,16,2011,1,1,매우 적음
1,2011-01-01 01:00:00,봄,0,0,1,9.02,13.63,80,0.0000,8,32,40,2011,1,1,매우 적음
2,2011-01-01 02:00:00,봄,0,0,1,9.02,13.63,80,0.0000,5,27,32,2011,1,1,매우 적음
3,2011-01-01 03:00:00,봄,0,0,1,9.84,14.39,75,0.0000,3,10,13,2011,1,1,매우 적음
4,2011-01-01 04:00:00,봄,0,0,1,9.84,14.39,75,0.0000,0,1,1,2011,1,1,매우 적음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,겨울,0,1,1,15.58,19.70,50,26.0027,7,329,336,2012,12,19,보통
10882,2012-12-19 20:00:00,겨울,0,1,1,14.76,17.43,57,15.0013,10,231,241,2012,12,19,보통
10883,2012-12-19 21:00:00,겨울,0,1,1,13.94,15.91,61,15.0013,4,164,168,2012,12,19,적음
10884,2012-12-19 22:00:00,겨울,0,1,1,13.94,17.43,61,6.0032,12,117,129,2012,12,19,적음


## 데이터전처리 과정을 마친 데이터프레임을 csv 형태로 저장

In [10]:
data.to_csv("99조_오득천.csv",encoding='euc-kr')